# Classifiers and training

Import required libraries.

In [1]:
import time
import scipy.io
import mne
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import antropy as ant
import EntropyHub as EH
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as clfrep
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

Load previously calculated entropies from files.

In [2]:
AE_normal_matrix = np.load("data/AE_normal_matrix.npy")
PE_normal_matrix = np.load("data/PE_normal_matrix.npy")
SE_normal_matrix = np.load("data/SE_normal_matrix.npy")
FE_normal_matrix = np.load("data/FE_normal_matrix.npy")

AE_fatigue_matrix = np.load("data/AE_fatigue_matrix.npy")
PE_fatigue_matrix = np.load("data/PE_fatigue_matrix.npy")
SE_fatigue_matrix = np.load("data/SE_fatigue_matrix.npy")
FE_fatigue_matrix = np.load("data/FE_fatigue_matrix.npy")

ENTROPIES = [
    [AE_normal_matrix, AE_fatigue_matrix], 
    [PE_normal_matrix, PE_fatigue_matrix], 
    [SE_normal_matrix, SE_fatigue_matrix], 
    [FE_normal_matrix, FE_fatigue_matrix]
]

ENTROPIES_NAMES = ["AE", "PE", "SE", "FE"]

num_entropies = 4
num_people = 12
num_epochs = 300
num_channels = 30

Clean NaN and infinity values from dataset.

In [3]:
for i in range(len(ENTROPIES)):
    for j in range(len(ENTROPIES[i])):
        np.nan_to_num(ENTROPIES[i][j], copy=False, nan=0.0, posinf=0.0, neginf=0.0)
        np.nan_to_num(ENTROPIES[i][j], copy=False, nan=0.0, posinf=0.0, neginf=0.0)

Generate dataset X using selected people. `train_index` contains indexes of people that will be used when creating the dataset.

In [4]:
def get_all_X(train_index):
    matrix_X = np.zeros(shape=(num_entropies, 2*num_epochs, num_channels))

    for person in train_index:
        for entropy in range(num_entropies):
            normal_X = np.zeros(shape=(num_epochs, num_channels))
            for ch in range(num_channels):
                for epoch in range(num_epochs):
                    normal_X[epoch][ch] = (ENTROPIES[entropy][0][person][epoch][ch][0])
            fatigue_X = np.zeros(shape=(num_epochs, num_channels))
            for ch in range(num_channels):
                for epoch in range(num_epochs):
                    fatigue_X[epoch][ch] = (ENTROPIES[entropy][1][person][epoch][ch][0])
                  
            # scale <-1, 1>
            scaler = MinMaxScaler((-1, 1))
            normal_scaled_X = scaler.fit_transform(normal_X)
            fatigue_scaled_X = scaler.fit_transform(fatigue_X)
            
            X = np.concatenate((normal_scaled_X, fatigue_scaled_X), axis=0)
            matrix_X[entropy] = X
        person_X = np.concatenate((matrix_X[0], matrix_X[1], matrix_X[2], matrix_X[3]), axis=1)
        if person == train_index[0]:
            matrix_all_X = person_X
        else:
            matrix_all_X = np.concatenate((matrix_all_X, person_X), axis=0)
        
    return matrix_all_X

Generate dataset y label using selected people. ```train_index``` contains indexes of people that will be used when creating the dataset.

In [5]:
def get_all_y(train_index):
    normal = list()
    fatigue = list()
    for epoch in range(num_epochs):
        normal.append(1)
    for epoch in range(num_epochs):
        fatigue.append(0)
    person_y = np.concatenate((normal, fatigue), axis=0)
            
    for person in train_index:
        if person == train_index[0]:
            people_y = person_y
        else:
            people_y = np.concatenate((people_y, person_y), axis=0)
            
    return people_y

Create subsets for multiple entropy. E.g.: (0, 1), (0, 2), (1, 2) ...

In [6]:
def create_subset(arr, r):
    return list(combinations(arr, r))

Split dataset 1:1 (50% / 50%) for training and testing. This dataset will be used in MLP, Random forest and K-neighbors classifiers.

In [7]:
people_indexes = [i for i in range(num_people)]

X = get_all_X(train_index=people_indexes)
y = get_all_y(train_index=people_indexes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

## MLP

In [8]:
clf = MLPClassifier(activation="logistic", hidden_layer_sizes=20, max_iter=500)
clf.fit(X_train, y_train)
clf.predict(X_test)
score = clf.score(X_test, y_test)
print(f"MLP score: {score}")

MLP score: 0.9711111111111111


/home/satan/.local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Random forest

In [9]:
clf = RandomForestClassifier(n_estimators=500, max_features=22)
clf.fit(X_train, y_train)
clf.predict(X_test)
score = clf.score(X_test, y_test)
print(f"RandomForest score: {score}")

RandomForest score: 0.9902777777777778


## K-neighbors

In [10]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train)
neigh.predict(X_test)
score = neigh.score(X_test, y_test)
print(f"K-neighbors score: {score}")

K-neighbors score: 0.9552777777777778


## SVM

### SVM using single entropy

In [11]:
# train model for each entropy
for entropy in range(num_entropies):
    for person in range(num_people):
        normal_X = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X[epoch][ch] = (ENTROPIES[entropy][0][person][epoch][ch][0])
        fatigue_X = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X[epoch][ch] = (ENTROPIES[entropy][1][person][epoch][ch][0])

        # scale <-1, 1>
        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X = scaler.fit_transform(normal_X)
        fatigue_scaled_X = scaler.fit_transform(fatigue_X)
        
        X = np.concatenate((normal_scaled_X, fatigue_scaled_X), axis=0)
        if person == 0:
            all_X = X
        else:
            all_X = np.concatenate((all_X, X), axis=0)
           
    X = np.array(all_X)
    
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    
    # train model
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entropy {ENTROPIES_NAMES[entropy]}")
    print(f"Score: {score}")
    print(clfrep_score)

Entropy AE
Score: 0.7736111111111111
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      1812
           1       0.81      0.71      0.76      1788

    accuracy                           0.77      3600
   macro avg       0.78      0.77      0.77      3600
weighted avg       0.78      0.77      0.77      3600

Entropy PE
Score: 0.8733333333333333
              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1812
           1       0.88      0.86      0.87      1788

    accuracy                           0.87      3600
   macro avg       0.87      0.87      0.87      3600
weighted avg       0.87      0.87      0.87      3600

Entropy SE
Score: 0.7569444444444444
              precision    recall  f1-score   support

           0       0.74      0.80      0.77      1812
           1       0.78      0.71      0.74      1788

    accuracy                           0.76      3600
   macro avg     

### SVM using two types of entropies

In [12]:
entropies_index_arr = [i for i in range(num_entropies)]
r = 2

entropy_subset_2 = create_subset(entropies_index_arr, r)
num_entropies_subsets = len(entropy_subset_2)
assert(num_entropies_subsets == 6)

entropy_subset_2_X = list()
entropy_X = np.zeros(shape=(num_people, 2 * num_epochs, 2 * num_channels))


# create entropy subsets
for entropy in entropy_subset_2:
    i = 0
    for person in range(num_people):
        normal_X1 = np.zeros(shape=(num_epochs, num_channels))    
        normal_X2 = np.zeros(shape=(num_epochs, num_channels))        
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X1[epoch][ch] = (ENTROPIES[entropy[0]][0][person][epoch][ch][0])
                normal_X2[epoch][ch] = (ENTROPIES[entropy[1]][0][person][epoch][ch][0])                
        fatigue_X1 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X2 = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X1[epoch][ch] = (ENTROPIES[entropy[0]][1][person][epoch][ch][0])
                fatigue_X2[epoch][ch] = (ENTROPIES[entropy[1]][1][person][epoch][ch][0])
      
        # scale <-1, 1>
        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X1 = scaler.fit_transform(normal_X1)
        normal_scaled_X2 = scaler.fit_transform(normal_X2)   
        fatigue_scaled_X1 = scaler.fit_transform(fatigue_X1)
        fatigue_scaled_X2 = scaler.fit_transform(fatigue_X2)

        X_1 = np.concatenate((normal_scaled_X1, fatigue_scaled_X1), axis=0)
        X_2 = np.concatenate((normal_scaled_X2, fatigue_scaled_X2), axis=0)
        
        X = np.concatenate((X_1, X_2), axis=1)
        entropy_X[i] = X
        i += 1
    
    for i in range(num_people):
        if i == 0:
            entropy_all_X = entropy_X[i]
        else:
            entropy_all_X = np.concatenate((entropy_all_X, entropy_X[i]), axis=0)
            
    entropy_subset_2_X.append(entropy_all_X)

    
assert(len(entropy_subset_2_X) == num_entropies_subsets)


# train model for each entropy subset
for i, X in enumerate(entropy_subset_2_X):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entropy {ENTROPIES_NAMES[entropy_subset_2[i][0]]}+{ENTROPIES_NAMES[entropy_subset_2[i][1]]}")
    print(f"Score: {score}")
    print(clfrep_score)

Entropy AE+PE
Score: 0.9294444444444444
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1812
           1       0.95      0.91      0.93      1788

    accuracy                           0.93      3600
   macro avg       0.93      0.93      0.93      3600
weighted avg       0.93      0.93      0.93      3600

Entropy AE+SE
Score: 0.8325
              precision    recall  f1-score   support

           0       0.81      0.87      0.84      1812
           1       0.86      0.80      0.83      1788

    accuracy                           0.83      3600
   macro avg       0.83      0.83      0.83      3600
weighted avg       0.83      0.83      0.83      3600

Entropy AE+FE
Score: 0.8522222222222222
              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1812
           1       0.89      0.80      0.84      1788

    accuracy                           0.85      3600
   macro avg       0

### SVM using three types of entropies

In [13]:
entropies_index_arr = [i for i in range(num_entropies)]
r = 3

entropy_subset_3 = create_subset(entropies_index_arr, r)
num_entropies_subsets = len(entropy_subset_3)
assert(num_entropies_subsets == 4)

entropy_subset_3_X = list()
entropy_X = np.zeros(shape=(num_people, 2 * num_epochs, 3 * num_channels))


# create entropy subsets
for entropy in entropy_subset_3:
    i = 0
    for person in range(num_people):
        normal_X1 = np.zeros(shape=(num_epochs, num_channels))    
        normal_X2 = np.zeros(shape=(num_epochs, num_channels))
        normal_X3 = np.zeros(shape=(num_epochs, num_channels))    
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X1[epoch][ch] = (ENTROPIES[entropy[0]][0][person][epoch][ch][0])
                normal_X2[epoch][ch] = (ENTROPIES[entropy[1]][0][person][epoch][ch][0])
                normal_X3[epoch][ch] = (ENTROPIES[entropy[2]][0][person][epoch][ch][0])
        fatigue_X1 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X2 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X3 = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X1[epoch][ch] = (ENTROPIES[entropy[0]][1][person][epoch][ch][0])
                fatigue_X2[epoch][ch] = (ENTROPIES[entropy[1]][1][person][epoch][ch][0])
                fatigue_X3[epoch][ch] = (ENTROPIES[entropy[2]][1][person][epoch][ch][0])

        # scale <-1, 1>
        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X1 = scaler.fit_transform(normal_X1)
        normal_scaled_X2 = scaler.fit_transform(normal_X2)
        normal_scaled_X3 = scaler.fit_transform(normal_X3)       
        fatigue_scaled_X1 = scaler.fit_transform(fatigue_X1)
        fatigue_scaled_X2 = scaler.fit_transform(fatigue_X2)
        fatigue_scaled_X3 = scaler.fit_transform(fatigue_X3)

        X_1 = np.concatenate((normal_scaled_X1, fatigue_scaled_X1), axis=0)
        X_2 = np.concatenate((normal_scaled_X2, fatigue_scaled_X2), axis=0)
        X_3 = np.concatenate((normal_scaled_X3, fatigue_scaled_X3), axis=0)
        
        X = np.concatenate((X_1, X_2, X_3), axis=1)
        entropy_X[i] = X
        i += 1
        
    for i in range(num_people):
        if i == 0:
            entropy_all_X = entropy_X[i]
        else:
            entropy_all_X = np.concatenate((entropy_all_X, entropy_X[i]), axis=0)
            
    entropy_subset_3_X.append(entropy_all_X)

    
assert(len(entropy_subset_3_X) == num_entropies_subsets)


# train model for each entropy subset
for i, X in enumerate(entropy_subset_3_X):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entropy {ENTROPIES_NAMES[entropy_subset_3[i][0]]}+{ENTROPIES_NAMES[entropy_subset_3[i][1]]}+{ENTROPIES_NAMES[entropy_subset_3[i][2]]}")
    print(f"Score: {score}")
    print(clfrep_score)

Entropy AE+PE+SE
Score: 0.9411111111111111
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1812
           1       0.95      0.93      0.94      1788

    accuracy                           0.94      3600
   macro avg       0.94      0.94      0.94      3600
weighted avg       0.94      0.94      0.94      3600

Entropy AE+PE+FE
Score: 0.9438888888888889
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      1812
           1       0.96      0.93      0.94      1788

    accuracy                           0.94      3600
   macro avg       0.94      0.94      0.94      3600
weighted avg       0.94      0.94      0.94      3600

Entropy AE+SE+FE
Score: 0.8844444444444445
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1812
           1       0.90      0.86      0.88      1788

    accuracy                           0.88      3600

### SVM using all four entropies

In [14]:
people_indexes = [i for i in range(num_people)]

# train on all people
X = get_all_X(train_index=people_indexes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
model = svc.fit(X_train, y_train)
h = model.predict(X_test)
score = accuracy_score(y_true=y_test, y_pred=h)
clfrep_score = clfrep(y_true=y_test, y_pred=h)

print(f"Entropy {'+'.join(ENTROPIES_NAMES)}")
print(f"Score: {score}")
print(clfrep_score)

Entropy AE+PE+SE+FE
Score: 0.9530555555555555
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1812
           1       0.96      0.94      0.95      1788

    accuracy                           0.95      3600
   macro avg       0.95      0.95      0.95      3600
weighted avg       0.95      0.95      0.95      3600



#### Grid search

In [15]:
people_indexes = [i for i in range(num_people)]

# train on all people
X = get_all_X(train_index=people_indexes)

C_list = [2**(-5), 2**(-1), 2**(5)]
gamma_list = [2**(-10), 2**(-5), 2**(10)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# grid search
for C in C_list:
    for gamma in gamma_list:
        svc = SVC(gamma=gamma, C=C, kernel="rbf")
        model = svc.fit(X_train, y_train)
        h = model.predict(X_test)
        score = accuracy_score(y_true=y_test, y_pred=h)
        clfrep_score = clfrep(y_true=y_test, y_pred=h)
        
        print(f"C = {C} gamma = {gamma} => score: {score}")
        print(clfrep_score)

C = 0.03125 gamma = 0.0009765625 => score: 0.5041666666666667
              precision    recall  f1-score   support

           0       0.94      0.02      0.03      1812
           1       0.50      1.00      0.67      1788

    accuracy                           0.50      3600
   macro avg       0.72      0.51      0.35      3600
weighted avg       0.72      0.50      0.35      3600

C = 0.03125 gamma = 0.03125 => score: 0.7736111111111111
              precision    recall  f1-score   support

           0       0.74      0.84      0.79      1812
           1       0.81      0.70      0.76      1788

    accuracy                           0.77      3600
   macro avg       0.78      0.77      0.77      3600
weighted avg       0.78      0.77      0.77      3600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


C = 0.03125 gamma = 1024 => score: 0.49666666666666665
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1812
           1       0.50      1.00      0.66      1788

    accuracy                           0.50      3600
   macro avg       0.25      0.50      0.33      3600
weighted avg       0.25      0.50      0.33      3600

C = 0.5 gamma = 0.0009765625 => score: 0.7097222222222223
              precision    recall  f1-score   support

           0       0.66      0.87      0.75      1812
           1       0.80      0.55      0.65      1788

    accuracy                           0.71      3600
   macro avg       0.73      0.71      0.70      3600
weighted avg       0.73      0.71      0.70      3600

C = 0.5 gamma = 0.03125 => score: 0.9530555555555555
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1812
           1       0.96      0.94      0.95      1788

    accuracy    

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


C = 0.5 gamma = 1024 => score: 0.49666666666666665
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1812
           1       0.50      1.00      0.66      1788

    accuracy                           0.50      3600
   macro avg       0.25      0.50      0.33      3600
weighted avg       0.25      0.50      0.33      3600

C = 32 gamma = 0.0009765625 => score: 0.8919444444444444
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1812
           1       0.91      0.87      0.89      1788

    accuracy                           0.89      3600
   macro avg       0.89      0.89      0.89      3600
weighted avg       0.89      0.89      0.89      3600

C = 32 gamma = 0.03125 => score: 0.9855555555555555
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1812
           1       0.99      0.98      0.99      1788

    accuracy          

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Grid search gives the best score of 0.98555 for C = 32 (`2**5`) and gamma = 0.03125 (`2**-5`).

## Leave-one-out (LOO) cross-validation

Training on eleven (out of twelve) people and testing on one person using all entropies.

In [16]:
people_indexes = [i for i in range(num_people)]

for C in C_list:
    for gamma in gamma_list:
        loo = LeaveOneOut()
        for train_index, test_index in loo.split(people_indexes):
            X_train = get_all_X(train_index)
            X_test= get_all_X(test_index)
            y_train = get_all_y(train_index)
            y_test = get_all_y(test_index)
            
            svc = SVC(gamma=gamma, C=C, kernel="rbf")
            model = svc.fit(X_train, y_train)
            h = model.predict(X_test)
            score = accuracy_score(y_true=y_test, y_pred=h)
            clfrep_score = clfrep(y_true=y_test, y_pred=h)
            
            print(f"test person = {test_index} C = {C} gamma = {gamma} => score: {score}")
            print(clfrep_score)

test person = [0] C = 0.03125 gamma = 0.0009765625 => score: 0.49
              precision    recall  f1-score   support

           0       0.47      0.13      0.21       300
           1       0.49      0.85      0.62       300

    accuracy                           0.49       600
   macro avg       0.48      0.49      0.42       600
weighted avg       0.48      0.49      0.42       600

test person = [1] C = 0.03125 gamma = 0.0009765625 => score: 0.69
              precision    recall  f1-score   support

           0       0.90      0.43      0.58       300
           1       0.62      0.95      0.75       300

    accuracy                           0.69       600
   macro avg       0.76      0.69      0.67       600
weighted avg       0.76      0.69      0.67       600

test person = [2] C = 0.03125 gamma = 0.0009765625 => score: 0.465
              precision    recall  f1-score   support

           0       0.41      0.16      0.23       300
           1       0.48      0.77     

test person = [9] C = 0.03125 gamma = 0.03125 => score: 0.5183333333333333
              precision    recall  f1-score   support

           0       0.51      0.97      0.67       300
           1       0.69      0.07      0.12       300

    accuracy                           0.52       600
   macro avg       0.60      0.52      0.39       600
weighted avg       0.60      0.52      0.39       600

test person = [10] C = 0.03125 gamma = 0.03125 => score: 0.27166666666666667
              precision    recall  f1-score   support

           0       0.35      0.51      0.41       300
           1       0.06      0.03      0.04       300

    accuracy                           0.27       600
   macro avg       0.20      0.27      0.23       600
weighted avg       0.20      0.27      0.23       600

test person = [11] C = 0.03125 gamma = 0.03125 => score: 0.225
              precision    recall  f1-score   support

           0       0.30      0.42      0.35       300
           1       0.0

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


test person = [0] C = 0.03125 gamma = 1024 => score: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


test person = [9] C = 32 gamma = 1024 => score: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


test person = [10] C = 32 gamma = 1024 => score: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600

test person = [11] C = 32 gamma = 1024 => score: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In paper the SVM classifier reached its optimized status for C = 0.5 (`2**-1`) and gamma = 0.03125 (`2**-5`).